In [51]:
import os
import glob
import re
import numpy as np
import numpy.fft as fft
import mne
import matplotlib.pyplot as plt
from scipy import stats, signal
from numpy import save, load
import pandas as pd
from sklearn.svm import SVC

### read dataset X and Y

In [63]:
subject_id=1
base_path = "features/"
edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
files=len(edf_file_names)
X=load('features/data_chb01/features_{}_00.npy'.format(subject_id))
y=load('features/data_chb01/targets_{}_00.npy'.format(subject_id))
print(X.shape)
for fileno in range(1, files//2):
    X=np.concatenate((X, load('features/data_chb01/features_{}_{:02d}.npy'.format(subject_id, fileno))))
    y=np.concatenate((y, load('features/data_chb01/targets_{}_{:02d}.npy'.format(subject_id, fileno))))


(408, 208)


In [64]:
X.shape, y.shape

((14584, 208), (14584,))

In [65]:
df=pd.DataFrame(data=X)

In [66]:
df['target']=y

In [68]:
df.shape

(14584, 209)

In [69]:
df.dropna()

,0,1,2,3,4,5,6,7,8,9,...,199,200,201,202,203,204,205,206,207,target
0,0.008798,0.204875,0.015422,0.016680,0.367948,0.065847,0.271457,0.048975,666.290633,0.008498,...,0.158389,0.005213,0.014143,0.358700,0.079680,0.306061,0.062087,138.171590,1.0,0
1,0.010540,0.192159,0.015494,0.017056,0.381521,0.075779,0.256403,0.051048,682.193538,0.008954,...,0.157175,0.007496,0.017795,0.370561,0.080321,0.291841,0.058733,130.508376,1.0,0
2,0.013913,0.199463,0.021406,0.022028,0.393250,0.070311,0.223988,0.055640,447.071529,0.010807,...,0.124585,0.009303,0.018426,0.401346,0.094435,0.262800,0.072888,126.237122,1.0,0
3,0.013022,0.211655,0.018351,0.022669,0.379469,0.062762,0.243671,0.048401,330.809577,0.020593,...,0.135654,0.009782,0.016507,0.397929,0.095829,0.251809,0.074261,119.865274,1.0,0
4,0.024974,0.175404,0.056200,0.058204,0.382374,0.061076,0.208994,0.032774,209.778695,0.046804,...,0.153252,0.012705,0.022622,0.305321,0.082453,0.328674,0.072447,97.171066,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14579,0.040064,0.170562,0.104434,0.082992,0.273975,0.057139,0.235965,0.034869,23.227600,0.025858,...,0.115704,0.020462,0.037875,0.264412,0.042061,0.398532,0.078592,52.704811,1.0,0
14580,0.039232,0.203132,0.022615,0.054827,0.319974,0.066159,0.259121,0.034939,23.717703,0.027324,...,0.120547,0.021694,0.041503,0.274336,0.041327,0.380874,0.075553,55.654383,1.0,0
14581,0.033145,0.213205,0.017147,0.044622,0.413378,0.052315,0.208488,0.017701,37.450236,0.107046,...,0.248857,0.055931,0.104552,0.337013,0.024445,0.101375,0.021093,19.298048,1.0,0
14582,0.033369,0.212081,0.016713,0.048256,0.403438,0.050923,0.217252,0.017967,38.988711,0.102688,...,0.250280,0.049431,0.109412,0.314262,0.026778,0.119075,0.023601,18.212296,1.0,0


In [70]:
df[df['target']==3]

,0,1,2,3,4,5,6,7,8,9,...,199,200,201,202,203,204,205,206,207,target
767,0.504019,0.161749,0.036241,0.087788,0.117071,0.010672,0.075750,0.006710,35.378636,0.340825,...,0.282103,0.070293,0.147826,0.065048,0.009173,0.059392,0.005359,22.022274,1.0,3
768,0.397320,0.131061,0.041063,0.131679,0.127953,0.018591,0.123891,0.028443,41.653349,0.137482,...,0.261818,0.062877,0.153241,0.082529,0.013847,0.073221,0.010040,24.051126,1.0,3
769,0.032970,0.067123,0.011638,0.045629,0.222607,0.082048,0.388199,0.149785,359.949214,0.022584,...,0.168848,0.029229,0.121630,0.179059,0.042258,0.250080,0.064458,80.585204,1.0,3
770,0.013406,0.067360,0.010145,0.044625,0.228867,0.084977,0.397326,0.153293,342.790860,0.017106,...,0.153139,0.030927,0.122526,0.180385,0.046731,0.282134,0.073164,88.751985,1.0,3
771,0.131704,0.184924,0.037553,0.238695,0.201855,0.023583,0.158190,0.023495,23.179371,0.198627,...,0.184617,0.045994,0.168155,0.154820,0.037461,0.221824,0.067128,31.520675,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9151,0.024826,0.082295,0.049787,0.065915,0.295219,0.086339,0.333806,0.061814,56.292130,0.102190,...,0.048700,0.042667,0.101418,0.115535,0.100833,0.517067,0.014999,97.952720,1.0,3
9152,0.014045,0.083170,0.027623,0.031817,0.319929,0.085996,0.365671,0.071750,75.450678,0.065839,...,0.056387,0.047088,0.089704,0.125709,0.098840,0.507692,0.015631,98.250833,1.0,3
9153,0.022369,0.120699,0.032243,0.047050,0.325650,0.063891,0.319887,0.068213,45.941663,0.062855,...,0.053789,0.052068,0.111783,0.115940,0.098990,0.508954,0.013975,97.544856,1.0,3
9154,0.034451,0.159178,0.060282,0.076829,0.325747,0.056455,0.239840,0.047217,27.680383,0.085372,...,0.045370,0.047184,0.114786,0.107574,0.100771,0.512889,0.013147,94.074203,1.0,3


In [76]:
df[df['target']!=3]
#discard discarded values

,0,1,2,3,4,5,6,7,8,9,...,199,200,201,202,203,204,205,206,207,target
0,0.008798,0.204875,0.015422,0.016680,0.367948,0.065847,0.271457,0.048975,666.290633,0.008498,...,0.158389,0.005213,0.014143,0.358700,0.079680,0.306061,0.062087,138.171590,1.0,0
1,0.010540,0.192159,0.015494,0.017056,0.381521,0.075779,0.256403,0.051048,682.193538,0.008954,...,0.157175,0.007496,0.017795,0.370561,0.080321,0.291841,0.058733,130.508376,1.0,0
2,0.013913,0.199463,0.021406,0.022028,0.393250,0.070311,0.223988,0.055640,447.071529,0.010807,...,0.124585,0.009303,0.018426,0.401346,0.094435,0.262800,0.072888,126.237122,1.0,0
3,0.013022,0.211655,0.018351,0.022669,0.379469,0.062762,0.243671,0.048401,330.809577,0.020593,...,0.135654,0.009782,0.016507,0.397929,0.095829,0.251809,0.074261,119.865274,1.0,0
4,0.024974,0.175404,0.056200,0.058204,0.382374,0.061076,0.208994,0.032774,209.778695,0.046804,...,0.153252,0.012705,0.022622,0.305321,0.082453,0.328674,0.072447,97.171066,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14579,0.040064,0.170562,0.104434,0.082992,0.273975,0.057139,0.235965,0.034869,23.227600,0.025858,...,0.115704,0.020462,0.037875,0.264412,0.042061,0.398532,0.078592,52.704811,1.0,0
14580,0.039232,0.203132,0.022615,0.054827,0.319974,0.066159,0.259121,0.034939,23.717703,0.027324,...,0.120547,0.021694,0.041503,0.274336,0.041327,0.380874,0.075553,55.654383,1.0,0
14581,0.033145,0.213205,0.017147,0.044622,0.413378,0.052315,0.208488,0.017701,37.450236,0.107046,...,0.248857,0.055931,0.104552,0.337013,0.024445,0.101375,0.021093,19.298048,1.0,0
14582,0.033369,0.212081,0.016713,0.048256,0.403438,0.050923,0.217252,0.017967,38.988711,0.102688,...,0.250280,0.049431,0.109412,0.314262,0.026778,0.119075,0.023601,18.212296,1.0,0


### train test split

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
clf = SVC()
clf.fit(X_train, y_train)
trainAcc = clf.score(X_train, y_train)
testAcc = clf.score(X_test, y_test)
print("**SVM Results:**")
print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")